In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit", # New Google 6 trillion tokens model 2.5x faster!
    "unsloth/gemma-2b-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-v0.3", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.8: Fast Mistral patching. Transformers: 4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.12.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
from datasets import load_dataset

# Load your dataset from Hugging Face
dataset = load_dataset("Nassira/MedAnalyzr", split="train")

# Define your custom prompt template
alpaca_prompt = """Vous êtes un médecin professionnel spécialisé dans l'analyse des résultats de laboratoire médical pour évaluer la santé des patients. Examinez les valeurs de test fournies, comparez-les aux plages de référence normales, et déterminez si elles sont normales ou anormales. Fournissez une évaluation diagnostique basée sur les anomalies. Priorisez toujours la précision et le professionnalisme dans votre réponse.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token  # Ensure to add EOS_TOKEN to avoid infinite generations

# Define the formatting function to prepare your data for fine-tuning
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    prompts      = examples["prompt"]  # Assuming 'prompt' is a field in your dataset
    texts = []

    for instruction, input, output, prompt in zip(instructions, inputs, outputs, prompts):
        # Format the prompt with instruction, input, output, and append EOS_TOKEN
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)

    return {"text": texts}

# Apply the formatting function to the dataset
dataset = dataset.map(formatting_prompts_func, batched=True)

# Now your dataset is ready for fine-tuning


README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

dataset (1).json:   0%|          | 0.00/2.32M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2182 [00:00<?, ? examples/s]

Map:   0%|          | 0/2182 [00:00<?, ? examples/s]

In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # Set num_train_epochs = 1 for full training runs
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/2182 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set num_train_epochs = 1 for full training runs
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [7]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,182 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 272
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.580200
2,1.592800
3,1.411500
4,1.175300
5,0.953100
6,0.719600
7,0.602100
8,0.636800
9,0.500000
10,0.572100


In [8]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Interpréter les résultats d'une analyse de gaz du sang, notamment le Base Excess (BE).", # instruction
        "{'CO2': 45, 'pH': 7.35, 'HCO3': 22, 'BE': 3}", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
tokenizer.batch_decode(outputs)

["<s> Vous êtes un médecin professionnel spécialisé dans l'analyse des résultats de laboratoire médical pour évaluer la santé des patients. Examinez les valeurs de test fournies, comparez-les aux plages de référence normales, et déterminez si elles sont normales ou anormales. Fournissez une évaluation diagnostique basée sur les anomalies. Priorisez toujours la précision et le professionnalisme dans votre réponse.\n\n### Instruction:\nInterpréter les résultats d'une analyse de gaz du sang, notamment le Base Excess (BE).\n\n### Input:\n{'CO2': 45, 'pH': 7.35, 'HCO3': 22, 'BE': 3}\n\n### Response:\nLe Base Excess (BE) est de 3 mEq/L, ce qui est dans la plage normale (0 à 2 mEq/L). Cela indique un équilibre parfait entre les acides et les bases dans le sang, ce qui est favorable pour la régulation de l'acidose et de l'alcalose.</s>"]

In [9]:
# Assuming the necessary imports are already done
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "Interpréter les résultats d'une analyse de gaz du sang, notamment le Base Excess (BE).",  # instruction
            "{'CO2': 45, 'pH': 7.35, 'HCO3': 22, 'BE': 3}",  # input
            "",  # output - leave this blank for generation!
        )
    ],
    return_tensors="pt"
).to("cuda")

outputs = model.generate(**inputs, max_new_tokens=128, use_cache=True)
decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

# Extract the response part
response_start = decoded_outputs.find("### Response:")
if response_start != -1:
    response = decoded_outputs[response_start + len("### Response:"):].strip()

print(response)


Le Base Excess (BE) est de 3 mEq/L, ce qui est dans la plage normale (0 à 2 mEq/L). Cela indique un équilibre parfait entre les acides et les bases dans le sang, ce qui est favorable pour la régulation de l'acidose et de l'alcalose.
